In [1]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect, Float
import joblib

In [2]:
engine = create_engine("sqlite:///credit_customer.db")

meta = MetaData()

customer= Table(
    'customer', meta, 
    Column('ID', String, primary_key=True), 
    Column('Customer_ID', String), 
    Column('Name', String),
    Column('Age', Integer), 
    Column('SSN', String), 
    Column('Occupation', String), 
    Column('Annual_Income', Float),
    Column('Monthly_Inhand_Salary', Float)        
)

In [3]:
credit= Table(
    'credit', meta, 
    Column('ID_Credit', Integer, primary_key=True),
    Column('ID', String, ForeignKey("customer.ID")), 
    Column('Num_Bank_Accounts', Integer),
    Column('Interest_Rate', Integer),
    Column('Num_of_Loan', Integer), 
    Column('Type_of_Loan', String),
    Column('Delay_from_due_date', Integer), 
    Column('Num_of_Delayed_Payment', Integer), 
    Column('Changed_Credit_Limit', Float),
    Column('Num_Credit_Inquiries', Float),
    Column('Credit_Mix', String),
    Column('Outstanding_Debt', Float),
    Column('Credit_Utilization_Ratio', Float), 
    Column('Payment_of_Min_Amount', String),
    Column('Total_EMI_per_month', Float),
    Column('Amount_invested_monthly', Float), 
    Column('Payment_Behaviour', String),
    Column('Monthly_Balance', Float),
    Column('Credit_History_Age_Years', Integer),
    Column('Credit_History_Age_Months', Integer)    
)

In [4]:
meta.create_all(engine)

In [5]:
df_customer = joblib.load('./df_customer.pkl')#file with clean data
values = list(zip(*map(df_customer.get, df_customer)))

In [6]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            # We indicate the format of a tuple of this table
            markers = ','.join('?' * len(values[0])) 
            # We use the SQL language in text format where markers is the format of a tuple
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            # This particular format is specified using the format member function
            ins = ins.format(tablename=customer.name, markers=markers)
            # Finally we can use the tuples created by executing the SQL command
            connection.execute(ins, values)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [7]:
df_credit = joblib.load('./df_credit.pkl')#file with clean data
values_credit = list(zip(*map(df_credit.get, df_credit)))

In [8]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            # We indicate the format of a tuple of this table
            markers = ','.join('?' * len(values_credit[0])) 
            # We use the SQL language in text format where markers is the format of a tuple
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            # This particular format is specified using the format member function
            ins = ins.format(tablename=credit.name, markers=markers)
            # Finally we can use the tuples created by executing the SQL command
            connection.execute(ins, values_credit)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

In [9]:
with engine.connect() as connection:
    results = connection.execute("SELECT * FROM customer LIMIT 1;")
    print(results.fetchall())

with engine.connect() as connection:
    results = connection.execute("SELECT * FROM credit LIMIT 1;")
    print(results.fetchall())

[('0x160a', 'CUS_0xd40', 'Aaron Maashoh', 23, '821-00-0265', 'Scientist', 19114.12, 1824.8433333333328)]
[(0, '0x160a', 3, 3, 19114, 'Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan', 3, 7, 11.27, 2022.0, 'Good', 809.98, 35.03040185583525, 'No', 49.57494921489417, 236.64268203272135, 'Low_spent_Small_value_payments', 186.26670208571772, 22, 9)]


In [10]:
#sql = text('DROP TABLE IF EXISTS customer;')
#result = engine.execute(sql)

In [11]:
#sql = text('DROP TABLE IF EXISTS credit;')
#result = engine.execute(sql)

In [12]:
conn = engine.connect() #Instantiate the Connection class in a variable named conn

In [13]:
joblib.dump(conn, './conn.pkl')

PicklingError: Can't pickle <function create_engine.<locals>.connect at 0x000001BB14874A60>: it's not found as sqlalchemy.engine.create.create_engine.<locals>.connect